In [ ]:
from matplotlib import pyplot as pl
import numpy as np
import pyneb as pn
from astropy.io.ascii import read

In [ ]:
# Recombination line Atom class
H1 = pn.RecAtom('H', 1)

# Collisionally-excited line Atom class
O3 = pn.Atom('O', 3)  # = [OIII]
O2 = pn.Atom('O', 2)  # = [OII]
S2 = pn.Atom('S', 2)  # = [SII]
N2 = pn.Atom('N', 2)  # = [NII]
Ne3 = pn.Atom('Ne', 3)  # = [NeIII]


In [ ]:
O3.printIonic()

In [ ]:
O3.printSources()

In [ ]:
print(pn.atomicData.getAllAvailableFiles('O3'))

In [ ]:
# default O3 data only goes to n=5, change to atmoic data that goes to n=6 for OIII]1666
pn.atomicData.setDataFile('o_iii_coll_AK99.dat')
O3 = pn.Atom('O', 3)

O3.printIonic()

In [ ]:
# the getEmissivity() function returns the emissivity of a transition for a given temperature and density

tem = 1.e4
den = 100.
hahb_int = H1.getEmissivity(tem, den, wave=6563)/H1.getEmissivity(tem, den, wave=4861)
print('Intrinsic Halpha/Hbeta = %.2f'%hahb_int)
hghb_int = H1.getEmissivity(tem, den, wave=4341)/H1.getEmissivity(tem, den, wave=4861)
print('Intrinsic Hgamma/Hbeta = %.2f'%hghb_int)

In [ ]:
temrange = np.linspace(5.e3, 2.e4, 1000)
oiii_5007_by_4363 = O3.getEmissivity(temrange, den, wave=5007)/O3.getEmissivity(temrange, den, wave=4363)
pl.plot(temrange, np.log10(oiii_5007_by_4363))
pl.gca().set_xlabel('T_e (K)')
pl.gca().set_ylabel('log([OIII]5007/[OIII]4363)')

In [ ]:
# some HII region line flux catalog from CHAOS survey (Berg+2015; Croxall+2015,2016; Berg+2020; Rogers+2021)
cat = read('hii_region_line_fluxes.dat')
print(list(cat.keys()))

In [ ]:
i = 25

# density from [SII]
s2rat = cat['sii6716_flux'].data[i]/cat['sii6731_flux'].data[i]
dens2 = S2.getTemDen(s2rat, tem=1.e4, wave1=6716, wave2=6731)
print('n_e([SII]) = %.0f cm^-3'%dens2)

# density of [OII]
o2rat = cat['oii3726_flux'].data[i]/cat['oii3729_flux'].data[i]
deno2 = O2.getTemDen(o2rat, tem=1.e4, wave1=3726, wave2=3729)
print('n_e([OII]) = %.0f cm^-3'%deno2)

In [ ]:
# temperature from [OIII]4363/5007 ratio
o3rat = cat['oiii4363_flux'].data[i]/cat['oiii5007_flux'].data[i]
temo3 = O3.getTemDen(o3rat, den=deno2, wave1=4363, wave2=5007)
print('T_e([OIII]) = %.0f K'%temo3)

In [ ]:
# temperature from [OII]7320,30
o2rat = (cat['oii7320_flux'].data[i] + cat['oii7330_flux'].data[i])/(cat['oii3726_flux'].data[i] + cat['oii3729_flux'].data[i])
temo2 = O2.getTemDen(o2rat, den=deno2, to_eval='(L(7320)+L(7330))/(L(3726)+L(3729))')
print('T_e([OII]) = %.0f K'%temo2)

# temperature from [NII]5755
n2rat = cat['nii5755_flux'].data[i]/cat['nii6584_flux'].data[i]
temn2 = N2.getTemDen(n2rat, den=deno2, wave1=5755, wave2=6584)
print('T_e([NII]) = %.0f K'%temn2)

In [ ]:
# adopt [OIII] temperature for the high ionization zone
temhigh = temo3
# adopt [OII] temperature for the high ionization zone
temlow = temn2

# calculate ionic O2+/H abundance ratios
o3hb = cat['oiii5007_flux'].data[i]/cat['hb_flux'].data[i]
o2p_hp = O3.getIonAbundance(o3hb, tem=temhigh, den=deno2, wave=5007, Hbeta=1.)
log_o2p_hp_12 = np.log10(o2p_hp) + 12.
print('12 + log(O2+/H+) = %.2f'%log_o2p_hp_12)

# alternative method using the emissivities directly
o3hbint = O3.getEmissivity(temhigh, deno2, wave=5007)/H1.getEmissivity(temhigh, deno2, wave=4861)
o2p_hp = o3hb/o3hbint
log_o2p_hp_12 = np.log10(o2p_hp) + 12.
print('Alternative log(O2+/H+) = %.2f'%log_o2p_hp_12)

# same result

In [ ]:
# O+/H abundance ratio
o2hb = (cat['oii3726_flux'].data[i] + cat['oii3729_flux'].data[i])/cat['hb_flux'].data[i]
op_hp = O2.getIonAbundance(o2hb, tem=temlow, den=deno2, to_eval='L(3726)+L(3729)', Hbeta=1.)
log_op_hp_12 = np.log10(op_hp) + 12.
print('12 + log(O+/H+) = %.2f'%log_op_hp_12)

In [ ]:
o_h = o2p_hp + op_hp
logoh12 = np.log10(o_h) + 12.
print('12 + log(O/H) = %.2f'%logoh12)

In [ ]:
# use emissivity ratios to calculate abundances N+/O+ and Ne2+/O2+
n2o2int = N2.getEmissivity(temlow, deno2, wave=6584)/(O2.getEmissivity(temlow, deno2, wave=3726) + O2.getEmissivity(temlow, deno2, wave=3729))
n2o2obs = cat['nii6584_flux'].data[i]/(cat['oii3726_flux'].data[i] + cat['oii3729_flux'].data[i])
np_op = n2o2obs/n2o2int
log_np_op = np.log10(np_op)
print('log(N+/O+) = %.2f'%log_np_op)

ne3o3int = Ne3.getEmissivity(temhigh, deno2, wave=3869)/O3.getEmissivity(temhigh, deno2, wave=5007)
ne3o3obs = cat['neiii3869_flux'].data[i]/cat['oiii5007_flux'].data[i]
ne2p_o2p = ne3o3obs/ne3o3int
log_ne2p_o2p = np.log10(ne2p_o2p)
print('log(Ne2+/O2+) = %.2f'%log_ne2p_o2p)

In [ ]:
# loop over all HII regions in the catalog
# calculate density and temperature
# calculate ionic abundances and O/H

logoh12arr = []
temo3arr = []
n2haarr = []
o3n2arr = []
o3hbarr = []
o2p_op_arr = []

for i in range(len(cat)):

 s2rat = cat['sii6716_flux'].data[i]/cat['sii6731_flux'].data[i]
 dens2 = S2.getTemDen(s2rat, tem=1.e4, wave1=6716, wave2=6731)

 o2rat = cat['oii3726_flux'].data[i]/cat['oii3729_flux'].data[i]
 deno2 = O2.getTemDen(o2rat, tem=1.e4, wave1=3726, wave2=3729)

 o3rat = cat['oiii4363_flux'].data[i]/cat['oiii5007_flux'].data[i]
 temo3 = O3.getTemDen(o3rat, den=deno2, wave1=4363, wave2=5007)

 o2rat = (cat['oii7320_flux'].data[i] + cat['oii7330_flux'].data[i])/(cat['oii3726_flux'].data[i] + cat['oii3729_flux'].data[i])
 temo2 = O2.getTemDen(o2rat, den=deno2, to_eval='(L(7320)+L(7330))/(L(3726)+L(3729))')

 n2rat = cat['nii5755_flux'].data[i]/cat['nii6584_flux'].data[i]
 temn2 = N2.getTemDen(n2rat, den=deno2, wave1=5755, wave2=6584)

 temhigh = temo3
 temlow = temn2

 o3hb = cat['oiii5007_flux'].data[i]/cat['hb_flux'].data[i]
 o2p_hp = O3.getIonAbundance(o3hb, tem=temhigh, den=deno2, wave=5007, Hbeta=1.)
 log_o2p_hp_12 = np.log10(o2p_hp) + 12.

 o3hbint = O3.getEmissivity(temhigh, deno2, wave=5007)/H1.getEmissivity(temhigh, deno2, wave=4861)
 o2p_hp = o3hb/o3hbint
 log_o2p_hp_12 = np.log10(o2p_hp) + 12.

 o2hb = (cat['oii3726_flux'].data[i] + cat['oii3729_flux'].data[i])/cat['hb_flux'].data[i]
 op_hp = O2.getIonAbundance(o2hb, tem=temlow, den=deno2, to_eval='L(3726)+L(3729)', Hbeta=1.)
 log_op_hp_12 = np.log10(op_hp) + 12.

 o_h = o2p_hp + op_hp
 logoh12 = np.log10(o_h) + 12.

 n2o2int = N2.getEmissivity(temlow, deno2, wave=6584)/(O2.getEmissivity(temlow, deno2, wave=3726) + O2.getEmissivity(temlow, deno2, wave=3729))
 n2o2obs = cat['nii6584_flux'].data[i]/(cat['oii3726_flux'].data[i] + cat['oii3729_flux'].data[i])
 np_op = n2o2obs/n2o2int
 log_np_op = np.log10(np_op)

 ne3o3int = Ne3.getEmissivity(temhigh, deno2, wave=3869)/O3.getEmissivity(temhigh, deno2, wave=5007)
 ne3o3obs = cat['neiii3869_flux'].data[i]/cat['oiii5007_flux'].data[i]
 ne2p_o2p = ne3o3obs/ne3o3int
 log_ne2p_o2p = np.log10(ne2p_o2p)

 logoh12arr += [logoh12]
 temo3arr += [temo3]
 n2haarr += [np.log10(cat['nii6584_flux'].data[i]/cat['ha_flux'].data[i])]
 o3n2arr += [np.log10(cat['nii6584_flux'].data[i]/cat['ha_flux'].data[i]) - np.log10(cat['oiii5007_flux'].data[i]/cat['hb_flux'].data[i])]
 o3hbarr += [np.log10(cat['oiii5007_flux'].data[i]/cat['hb_flux'].data[i])]
 o2p_op_arr += [o2p_hp/op_hp]

logoh12arr = np.array(logoh12arr)
temo3arr = np.array(temo3arr)
n2haarr = np.array(n2haarr)
o3n2arr = np.array(o3n2arr)
o3hbarr = np.array(o3hbarr)
o2p_op_arr = np.array(o2p_op_arr)

pl.scatter(n2haarr, o3hbarr, c=logoh12arr)
cbar = pl.colorbar()
cbar.set_label('12+log(O/H)')
pl.gca().set_title('[NII] BPT diagram')
pl.gca().set_xlabel('log([NII]6584/Halpha)')
pl.gca().set_ylabel('log([OIII]5007/Hbeta)')
pl.show()

pl.scatter(logoh12arr, temo3arr)
pl.gca().set_title('T_e vs. metallicity')
pl.gca().set_xlabel('12+log(O/H)')
pl.gca().set_ylabel('T_e([OIII])')
pl.show()
